In [64]:
import pandas as pd
import json

In [65]:
with open('data.json') as file:
    data = json.loads(file.read())

In [43]:

def raw_json_to_df(json):
    return pd.DataFrame(json)

def bot_to_df(days_dict: dict):
    df = pd.concat([raw_json_to_df(data) for date, data in days_dict.items()])
    df = df[~df['symbol'].isna()]
    return df.reset_index(drop = True)

def data_to_bot_dfs(raw_data: dict):
    return {bot: bot_to_df(data) for bot, data in raw_data.items()}

def format_buy_sell_df(buy_sell_df: pd.DataFrame) -> pd.DataFrame:

    # Convert entry_time and exit_time to datetime objects
    buy_sell_df['entry_time'] = pd.to_datetime(buy_sell_df['entry_time'])
    buy_sell_df['exit_time'] = pd.to_datetime(buy_sell_df['exit_time'])

    # Convert prices and qty to numeric values
    buy_sell_df['bought_price'] = buy_sell_df['bought_price'].astype(float)
    buy_sell_df['sold_price'] = buy_sell_df['sold_price'].astype(float)
    buy_sell_df['qty'] = buy_sell_df['qty'].astype(int)

    # Calculate trade duration
    buy_sell_df['trade_duration'] = buy_sell_df['exit_time'] - buy_sell_df['entry_time']

    # Calculate profit percentage, profit amount, and gain or loss
    buy_sell_df['profit_pct'] = ((buy_sell_df['sold_price'] - buy_sell_df['bought_price']) / buy_sell_df['bought_price']) * 100
    buy_sell_df['profit_amount'] = (buy_sell_df['sold_price'] - buy_sell_df['bought_price']) * buy_sell_df['qty']
    buy_sell_df['Gain_or_Loss'] = buy_sell_df['profit_amount'].apply(lambda x: 'Gain' if x > 0 else 'Loss')

    return buy_sell_df


def remove_unequated_orders(df: pd.DataFrame) -> pd.DataFrame:

    # starting here shouldnt be neccessary if you have all the bot trades. Only needed if it is examining from the middle where there can be exits or entries alone.

    rows_to_remove = []

    for symbol in df['symbol'].unique():
        symbol_rows = df[df['symbol'] == symbol]
        
        if symbol_rows['filled_qty'].iloc[:2].nunique() > 1:
            rows_to_remove.append(symbol_rows.index[0])
        
        # Check the last 2 quantities
        if symbol_rows['filled_qty'].iloc[-2:].nunique() > 1:
            rows_to_remove.append(symbol_rows.index[-1])

    # Drop the identified rows
    df = df.drop(rows_to_remove)

    symbol_counts = df['symbol'].value_counts()
    symbols_to_remove = symbol_counts[symbol_counts == 1].index.tolist()
    # Remove rows for symbols with only one row
    df = df[~df['symbol'].isin(symbols_to_remove)]
    print(symbols_to_remove)
    # until here should not be necessary

    return df


def df_to_buy_sell_df(df: pd.DataFrame) -> pd.DataFrame:

    buy_sell_df = pd.DataFrame(columns=['symbol', 'bought_price', 'sold_price', 'qty', 'side', 'entry_time', 'exit_time'])
    
    df = remove_unequated_orders(df) # this should not be requried

    rows_to_ignore = []

    for index, row in df.iterrows():

        if index in rows_to_ignore:
            continue

        symbol = row['symbol']
        side = row['side']
        qty = float(row['filled_qty'])
        entry_time = row['filled_at']
        entry_price = row['filled_avg_price']
        # print(symbol)
        # Find the next occurrence (exit) of the symbol with opposite side
        exit_row = df[(df['symbol'] == symbol) & (df['side'] != side) & (df['filled_at'] > entry_time)].iloc[0]
        mathched_qty = float(exit_row['filled_qty'])
        current_iloc = 0

        while mathched_qty < qty:
            print(1)
            current_iloc += 1
            new_row = df[(df['symbol'] == symbol) & (df['side'] != side) & (df['filled_at'] > entry_time)].iloc[current_iloc]
            mathched_qty += float(new_row['filled_qty'])
            exit_row['filled_avg_price'] = (float(exit_row['filled_avg_price']) * float(exit_row['filled_qty']) + float(new_row['filled_avg_price']) * float(new_row['filled_qty'])) / mathched_qty
            exit_row['filled_qty'] = mathched_qty
            rows_to_ignore.append(new_row.name) # adding extra row to ignore
            # print(new_row['symbol'])

        exit_time = exit_row['filled_at']
        exit_price = str(exit_row['filled_avg_price'])
        
        # Append to buy_sell_df
        buy_sell_pair = pd.DataFrame([{
                'symbol': symbol,
                'bought_price': entry_price if side == 'buy' else exit_price,
                'sold_price': exit_price if side == 'buy' else entry_price,
                'qty': str(qty),
                'side': side,
                'entry_time': entry_time,
                'exit_time': exit_time
            }])
            
            # Concatenate the buy-sell pair DataFrame to buy_sell_df
        buy_sell_df = pd.concat([buy_sell_df, buy_sell_pair], ignore_index=True)


        # Making exit row get skipped by loop
        exit_row_index = exit_row.name
        rows_to_ignore.append(exit_row_index)
        rows_to_ignore.append(index)

    buy_sell_df = format_buy_sell_df(buy_sell_df=buy_sell_df)

    return buy_sell_df



In [100]:
import pandas as pd

def raw_json_to_df(json):
    df = pd.DataFrame(json)
    df = df[~df['side'].isna()]
    # df['filled_at'] = pd.to_datetime(df['filled_at'], unit='ms')  # Convert Unix timestamp to datetime
    return df

def bot_to_df(days_dict: dict):
    df = pd.concat([raw_json_to_df(data) for date, data in days_dict.items() if date != 'delete'])
    return df.reset_index(drop=True)

def format_buy_sell_df(buy_sell_df: pd.DataFrame) -> pd.DataFrame:
    # Convert entry_time and exit_time to datetime objects
    # buy_sell_df['entry_time'] = pd.to_datetime(buy_sell_df['entry_time'], unit='ms')
    # buy_sell_df['exit_time'] = pd.to_datetime(buy_sell_df['exit_time'], unit='ms')

    # Convert prices and qty to numeric values
    buy_sell_df['bought_price'] = buy_sell_df['bought_price'].astype(float)
    buy_sell_df['sold_price'] = buy_sell_df['sold_price'].astype(float)
    buy_sell_df['qty'] = buy_sell_df['qty'].astype(float)

    # Calculate trade duration
    buy_sell_df['trade_duration'] = buy_sell_df['exit_time'] - buy_sell_df['entry_time']

    # Calculate profit percentage, profit amount, and gain or loss
    buy_sell_df['profit_pct'] = ((buy_sell_df['sold_price'] - buy_sell_df['bought_price']) / buy_sell_df['bought_price']) * 100
    buy_sell_df['profit_amount'] = (buy_sell_df['sold_price'] - buy_sell_df['bought_price']) * buy_sell_df['qty']
    buy_sell_df['Gain_or_Loss'] = buy_sell_df['profit_amount'].apply(lambda x: 'Gain' if x > 0 else 'Loss')

    return buy_sell_df

def remove_unequated_orders(df: pd.DataFrame) -> pd.DataFrame:

    # starting here shouldnt be neccessary if you have all the bot trades. Only needed if it is examining from the middle where there can be exits or entries alone.

    rows_to_remove = []

    for symbol in df['symbol'].unique():
        symbol_rows = df[df['symbol'] == symbol]

        if symbol_rows['filled_qty'].iloc[:2].nunique() > 1:
            rows_to_remove.append(symbol_rows.index[0])

        # Check the last 2 quantities
        if symbol_rows['filled_qty'].iloc[-2:].nunique() > 1:
            rows_to_remove.append(symbol_rows.index[-1])

    # Drop the identified rows
    df = df.drop(rows_to_remove)

    symbol_counts = df['symbol'].value_counts()
    symbols_to_remove = symbol_counts[symbol_counts == 1].index.tolist()
    # Remove rows for symbols with only one row
    df = df[~df['symbol'].isin(symbols_to_remove)]

    print(f"Unequated stocks: {', '.join(symbols_to_remove)}")

    # until here should not be necessary

    return df

def concat_rows(row: pd.Series, df_continuation: pd.DataFrame, qty_to_match: float = None) -> list[float, float, int]:
    """
    This function takes a row and the rows from the ticker after to see if there are multiple entries in a row.
    If there are multiple entries in a row (AKA the side is the same 2+ times in a row, it combines the row into one)
    This works for both finding the entry order total and the exit order total.
    The function returns the qty and the price as those are the only things that change.

    Args:
        row (pd.Series): the current row to find additional add on orders for.
        df_continuation (pd.DataFrame): the continuation of the stock df.
        qty_to_match (float): If it is an exit, there could be a case where there is a buy, sell, sell, but and we don't want to 
        interpret that as a buy, (sel, sell), buy... so we need to give the qty limit so that it knows when it is a new order.

    Returns:
        list[float, float, int]: the qty, price, and the current iloc to know which rows to ignore. 
        If the iloc is equal to 0 it means that no additional rows should be removed (current_iloc = amount of rows to remove)
    """

    side, qty, price = (row['side'], float(row['filled_qty']), float(row['filled_avg_price']))
    
    df_continuation['filled_qty'] = df_continuation['filled_qty'].astype(float)
    df_continuation['filled_avg_price'] = df_continuation['filled_avg_price'].astype(float)

    current_iloc = 0
    lastest_fill = row['filled_at']

    while True:

        if qty_to_match is not None:
            if qty >= qty_to_match: # this should never be bigger and should always be equal but there is a raise in the function that calls this function to deal witht that.
                break
        
        try:
            current_row = df_continuation.iloc[current_iloc]
        except IndexError:
            break

        if current_row['side'] != side:
            break
        else:
            price = (price * qty + current_row['filled_qty'] * current_row['filled_avg_price'])
            qty += current_row['filled_qty']
            price /= qty
            
            lastest_fill = current_row['filled_at']
            current_iloc += 1

    return qty, price, current_iloc, lastest_fill
            
def df_to_buy_sell_df(df: pd.DataFrame) -> pd.DataFrame:

    buy_sell_df = pd.DataFrame(columns=['symbol', 'bought_price', 'sold_price', 'qty', 'side', 'entry_time', 'exit_time'])

    # df = remove_unequated_orders(df) # this should not be requried
    df = df.sort_values('filled_at')
    rows_to_ignore = []

    for index, row in df.iterrows():

        if index in rows_to_ignore:
            continue

        symbol = row['symbol']
        side = row['side']

        next_rows: pd.DataFrame = df[(df['symbol'] == symbol) & (df['filled_at'] > row['filled_at'])]

        qty, entry_price, rows_to_remove, entry_time = concat_rows(row, next_rows)

        # Ignoring rows that were concatenated with the entry due to samw side.
        if rows_to_remove != 0:
            for i in range(rows_to_remove):
                rows_to_ignore.append(next_rows.iloc[i].name)

      
        # Find the next occurrence (exit) of the symbol with opposite side
        try:
            exit_row = df[(df['symbol'] == symbol) & (df['filled_at'] > entry_time)].iloc[0]
        except:
            return buy_sell_df
        
        if exit_row['side'] == side:
            raise Exception("This should not be possible as all the same sided orders should have been concatenated and then ignored. Debug.")
        
        next_rows_exit: pd.DataFrame = df[(df['symbol'] == symbol) & (df['filled_at'] > exit_row['filled_at'])]
        exit_qty, exit_price, exit_rows_to_remove, exit_time = concat_rows(exit_row, next_rows_exit, qty_to_match=qty)

        if exit_qty != qty:
            print(symbol)
            print(qty)
            print(exit_qty)
            raise Exception("Unequated order: to entry qty does not match up with the exit qty. Debug.")
        
        if exit_rows_to_remove != 0:
            for i in range(rows_to_remove):
                rows_to_ignore.append(next_rows_exit.iloc[i].name)

        # Append to buy_sell_df
        buy_sell_pair = pd.DataFrame([{
                'symbol': symbol,
                'bought_price': entry_price if side == 'buy' else exit_price,
                'sold_price': exit_price if side == 'buy' else entry_price,
                'qty': qty,
                'side': side,
                'entry_time': entry_time,
                'exit_time': exit_time
            }])

            # Concatenate the buy-sell pair DataFrame to buy_sell_df
        buy_sell_df = pd.concat([buy_sell_df, buy_sell_pair], ignore_index=True)


        # Making exit row get skipped by loop
        exit_row_index = exit_row.name
        rows_to_ignore.append(exit_row_index)
        rows_to_ignore.append(index)

    buy_sell_df = format_buy_sell_df(buy_sell_df=buy_sell_df)

    return buy_sell_df

def convert_to_buy_sell(raw_json):

    user_df = bot_to_df(raw_json)
    user_buy_sell_df = df_to_buy_sell_df(user_df)
    return user_buy_sell_df.to_dict(orient='records')


In [101]:
bot_dfs = data_to_bot_dfs(data)

In [107]:
df = bot_dfs['mordechaidaniel0']
df[df['symbol'] == 'ARM']
df[df['symbol'] == 'BMR']
df.count()/2

filled_at           148.5
filled_avg_price    148.5
filled_qty          148.5
side                148.5
symbol              148.5
type                148.5
dtype: float64

In [104]:
bot_buy_sell_dfs = {bot: df_to_buy_sell_df(df) for bot, df in bot_dfs.items()}

C:\Users\morde\AppData\Local\Temp\ipykernel_16764\2709123664.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_continuation['filled_qty'] = df_continuation['filled_qty'].astype(float)
C:\Users\morde\AppData\Local\Temp\ipykernel_16764\2709123664.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_continuation['filled_avg_price'] = df_continuation['filled_avg_price'].astype(float)
C:\Users\morde\AppData\Local\Temp\ipykernel_16764\2709123664.py:83: SettingWithCopyWarning: 
A value is trying to be set

In [110]:
pd.set_option('display.max_rows', 138)
bot_buy_sell_dfs['mordechaidaniel0']

,symbol,bought_price,sold_price,qty,side,entry_time,exit_time
0,CLSK,16.080000,15.940000,2487.0,buy,1.707850e+12,1.707850e+12
1,CLSK,15.930000,15.780000,2507.0,buy,1.707850e+12,1.707851e+12
2,CLSK,15.812787,15.650000,2530.0,buy,1.707851e+12,1.707852e+12
3,SATS,12.500000,12.390000,3200.0,buy,1.707851e+12,1.707855e+12
4,CVNA,52.200000,51.671292,766.0,buy,1.707852e+12,1.707854e+12
5,COIN,141.530000,140.030000,282.0,buy,1.707852e+12,1.707854e+12
6,NOVA,10.710000,10.600000,3734.0,buy,1.707852e+12,1.707856e+12
7,CLSK,15.990000,15.830000,2526.0,sell,1.707852e+12,1.707853e+12
8,SEDG,76.770000,76.450000,521.0,buy,1.707854e+12,1.707858e+12
9,CVNA,51.466538,51.610000,777.0,buy,1.707854e+12,1.707858e+12


In [58]:
test = bot_buy_sell_dfs['mordechaidaniel0']
test[test['symbol'] == 'ARM']

,symbol,bought_price,sold_price,qty,side,entry_time,exit_time,trade_duration,profit_pct,profit_amount,Gain_or_Loss
23,ARM,129.380000,128.050000,300.0,buy,1.707921e+12,1.707921e+12,55789.0,-1.027980,-399.000000,Loss
33,ARM,127.279967,129.700000,606.0,buy,1.708008e+12,1.708009e+12,683002.0,1.901346,1466.539998,Gain
55,ARM,132.450000,133.490000,582.0,buy,1.708030e+12,1.708031e+12,420754.0,0.785202,605.280000,Gain
107,ARM,118.610000,120.920000,692.0,buy,1.708448e+12,1.708453e+12,4539450.0,1.947559,1598.520000,Gain
111,ARM,120.760318,121.408964,691.0,buy,1.708456e+12,1.708463e+12,6596135.0,0.537135,448.214386,Gain
112,ARM,120.770000,121.408964,679.0,buy,1.708456e+12,1.708463e+12,6594570.0,0.529075,433.856556,Gain


In [57]:
df.groupby('symbol')['symbol'].count()

symbol
AAOI     2
AFRM     8
ARM     11
AS       2
BMR      8
CHWY     4
CLSK    46
COIN     8
CVNA    16
CYTK     2
DYN      4
ENPH     4
ENVX     2
FSLY     2
GCT      2
GDXU     2
HUT      4
IOVA     4
IRBT     4
LBPH     2
MARA    31
MBLY     2
MRNA     8
MSTR     2
NOVA    30
PACB     2
PARA     3
RILY     4
RIVN     6
ROKU     2
RUN      8
RXRX     2
SATS     2
SEDG     2
SMCI     2
SNAP     6
SOFI     2
SOXL     4
TGTX     2
TSLT     8
TZA      6
U        2
UNG      4
UVIX    10
VKTX     4
WOLF     4
ZIM      2
Name: symbol, dtype: int64

In [12]:
df_to_buy_sell_df(para_df)

,symbol,bought_price,sold_price,qty,side,entry_time,exit_time,trade_duration,profit_pct,profit_amount,Gain_or_Loss


In [13]:
para_df

,filled_at,filled_avg_price,filled_qty,side,symbol,type
276,1.708541e+12,11.81,6884,buy,PARA,market
291,1.708549e+12,11.77,6102,sell,PARA,market
296,1.708549e+12,11.77,782,sell,PARA,market


In [111]:
json_dict = {key: df.to_json(orient='records') for key, df in bot_buy_sell_dfs.items()}

In [112]:
json_dict

{'Bot 1': '[{"symbol":"LYFT","bought_price":10.33,"sold_price":10.37,"qty":3857,"side":"sell","entry_time":1700679,"exit_time":1700686,"trade_duration":7,"profit_pct":0.3872216844,"profit_amount":154.28,"Gain_or_Loss":"Gain"},{"symbol":"AGL","bought_price":11.75,"sold_price":11.630141,"qty":3404,"side":"buy","entry_time":1700681,"exit_time":1700681,"trade_duration":0,"profit_pct":-1.0200765957,"profit_amount":-408.000036,"Gain_or_Loss":"Loss"},{"symbol":"MARA","bought_price":10.5,"sold_price":10.712518,"qty":3809,"side":"buy","entry_time":1700681,"exit_time":1700682,"trade_duration":1,"profit_pct":2.0239809524,"profit_amount":809.481062,"Gain_or_Loss":"Gain"},{"symbol":"SOXL","bought_price":23.25,"sold_price":23.03,"qty":1719,"side":"buy","entry_time":1700681,"exit_time":1700681,"trade_duration":0,"profit_pct":-0.9462365591,"profit_amount":-378.18,"Gain_or_Loss":"Loss"},{"symbol":"PLTR","bought_price":19.68,"sold_price":19.66,"qty":2032,"side":"buy","entry_time":1700681,"exit_time":170

In [113]:
bot_buy_sell_dfs['Bot 1'].to_json(orient='records')

'[{"symbol":"LYFT","bought_price":10.33,"sold_price":10.37,"qty":3857,"side":"sell","entry_time":1700679,"exit_time":1700686,"trade_duration":7,"profit_pct":0.3872216844,"profit_amount":154.28,"Gain_or_Loss":"Gain"},{"symbol":"AGL","bought_price":11.75,"sold_price":11.630141,"qty":3404,"side":"buy","entry_time":1700681,"exit_time":1700681,"trade_duration":0,"profit_pct":-1.0200765957,"profit_amount":-408.000036,"Gain_or_Loss":"Loss"},{"symbol":"MARA","bought_price":10.5,"sold_price":10.712518,"qty":3809,"side":"buy","entry_time":1700681,"exit_time":1700682,"trade_duration":1,"profit_pct":2.0239809524,"profit_amount":809.481062,"Gain_or_Loss":"Gain"},{"symbol":"SOXL","bought_price":23.25,"sold_price":23.03,"qty":1719,"side":"buy","entry_time":1700681,"exit_time":1700681,"trade_duration":0,"profit_pct":-0.9462365591,"profit_amount":-378.18,"Gain_or_Loss":"Loss"},{"symbol":"PLTR","bought_price":19.68,"sold_price":19.66,"qty":2032,"side":"buy","entry_time":1700681,"exit_time":1700686,"trad

In [74]:
len(bot_dfs['Bot 1'])

38

In [75]:
len(bot_buy_sell_dfs['Bot 1']) * 2

38

In [76]:
bot_dfs['Bot 1']

,filled_at,filled_avg_price,filled_qty,side,symbol,type
0,1700679275108,10.37,3857,sell,LYFT,market
1,1700681178729,11.75,3404,buy,AGL,market
2,1700681324996,10.5,3809,buy,MARA,market
3,1700682910364,10.712518,3809,sell,MARA,limit
4,1700681460155,23.25,1719,buy,SOXL,market
5,1700681481372,19.68,2032,buy,PLTR,market
6,1700681862426,23.03,1719,sell,SOXL,stop
7,1700681905184,11.630141,3404,sell,AGL,stop
8,1700681902660,13.915283,2875,buy,TOST,market
9,1700684303726,11.75,3404,sell,AGL,market


In [77]:
bot_dfs['Bot 1'].groupby(['symbol']).count()

,filled_at,filled_avg_price,filled_qty,side,type
symbol,,,,,
AGL,6,6,6,6,6
LYFT,4,4,4,4,4
MARA,4,4,4,4,4
NOVA,4,4,4,4,4
PLTR,6,6,6,6,6
RIVN,2,2,2,2,2
RUN,4,4,4,4,4
SOXL,2,2,2,2,2
TOST,2,2,2,2,2
